In [2]:
def generate_structural_diagram_with_slabs(obj_file, grid_size=6, floor_height=4, num_floors=3):
    """
    Generates a structural diagram with:
    - Core walls (forming a closed box)
    - Optimized column placement (around cores and perimeter)
    - Beams forming a logical structural grid
    - Slabs at each floor level
    """
    # Load the 3D model
    mesh = trimesh.load_mesh(obj_file)

    # Compute bounding box for grid & structure
    bounds = mesh.bounds
    min_x, min_y, min_z = bounds[0]
    max_x, max_y, max_z = bounds[1]

    # Detect Core Area: Bounding Box of Lower 20% of Mass
    core_z_threshold = (max_z - min_z) * 0.2
    core_mask = mesh.vertices[:, 2] < min_z + core_z_threshold
    core_points = mesh.vertices[core_mask]

    # Create a Tight Bounding Box Around Core Area
    if len(core_points) > 3:
        core_min_x, core_min_y = np.min(core_points[:, :2], axis=0)
        core_max_x, core_max_y = np.max(core_points[:, :2], axis=0)
        core_corners = [
            (core_min_x, core_min_y, min_z), (core_max_x, core_min_y, min_z),
            (core_max_x, core_max_y, min_z), (core_min_x, core_max_y, min_z)
        ]
        core_walls = [(core_corners[i], core_corners[(i + 1) % 4]) for i in range(4)]
    else:
        core_walls = []

    # Generate Column Grid: Place at Core Corners & Perimeter
    x_positions = np.arange(min_x, max_x + 1, grid_size)
    y_positions = np.arange(min_y, max_y + 1, grid_size)
    columns = [(x, y, min_z) for x in x_positions for y in y_positions
               if (x <= core_min_x or x >= core_max_x) or (y <= core_min_y or y >= core_max_y)]

    # Extend Columns Vertically (Multi-Floor)
    full_columns = [(x, y, z) for x, y, _ in columns for z in np.arange(min_z, max_z, floor_height)]

    # Define Beams Between Columns (Structurally Logical Connections)
    beams = []
    for col in columns:
        x, y, z = col
        neighbors = [(x + grid_size, y, z), (x, y + grid_size, z)]
        for n in neighbors:
            if n in columns:
                beams.append((col, n))

    # Define Slabs (Horizontal Planes at Each Floor Level)
    slabs = []
    for floor in range(1, num_floors + 1):
        slab_z = min_z + floor * floor_height
        slab_corners = [
            (min_x, min_y, slab_z), (max_x, min_y, slab_z),
            (max_x, max_y, slab_z), (min_x, max_y, slab_z)
        ]
        slabs.append(slab_corners)

    # 📌 Structural Visualization in 3D
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot Core Walls (Closed Box)
    for wall in core_walls:
        p1, p2 = wall
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [min_z, min_z + max_z * 0.6],
                color='orange', lw=3, label='Core Walls' if wall == core_walls[0] else "")

    # Plot Columns (Optimized)
    for col in full_columns:
        ax.scatter(*col, color='r', s=50, label='Columns' if col == full_columns[0] else "")

    # Plot Beams (Connecting Structural Grid)
    for beam in beams:
        p1, p2 = beam
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='b', lw=1.5,
                label='Beams' if beam == beams[0] else "")

    # Plot Slabs (Horizontal Layers)
    for slab in slabs:
        x_vals = [p[0] for p in slab] + [slab[0][0]]
        y_vals = [p[1] for p in slab] + [slab[0][1]]
        z_vals = [p[2] for p in slab] + [slab[0][2]]
        ax.plot(x_vals, y_vals, z_vals, color='purple', lw=2, linestyle='dotted', label='Slabs' if slab == slabs[0] else "")

    # Labels & Display
    ax.set_xlabel("X Axis (m)")
    ax.set_ylabel("Y Axis (m)")
    ax.set_zlabel("Height (m)")
    plt.legend()
    plt.title("Optimized Structural Diagram with Slabs & Improved Column Placement")
    plt.show()

# Run the improved structural visualization
generate_structural_diagram_with_slabs("/mnt/data/sample_model.obj")

NameError: name 'trimesh' is not defined

In [ ]:
%pip install PyNite

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from PyNite import FEModel3D

# Create a new finite element model
model = FEModel3D()

# Define a default material (can be customized)
model.add_material('Steel', E=29000, G=11500, nu=0.3, rho=0.284)

# Create nodes from unique coordinates
model.add_node('N1', -7.17073392868042, 6.841569900512695, 0) # Also represents: N15, N17
model.add_node('N2', -7.17073392868042, 6.841569900512695, 5) # Also represents: N16, N21
model.add_node('N3', 1.2679941654205322, 6.841569900512695, 0) # Also represents: N5, N18, N25, N39, N41
model.add_node('N4', 1.2679941654205322, 6.841569900512695, 5) # Also represents: N6, N22, N26, N40, N45
model.add_node('N7', 1.2679941654205322, 11.352361679077148, 0) # Also represents: N9, N19
model.add_node('N8', 1.2679941654205322, 11.352361679077148, 5) # Also represents: N10, N23
model.add_node('N11', -7.17073392868042, 11.352361679077148, 0) # Also represents: N13, N20
model.add_node('N12', -7.17073392868042, 11.352361679077148, 5) # Also represents: N14, N24
model.add_node('N27', 3.2604687213897705, 6.841569900512695, 0) # Also represents: N29, N42, N151, N153, N163
model.add_node('N28', 3.2604687213897705, 6.841569900512695, 5) # Also represents: N30, N46, N152, N154, N167
model.add_node('N31', 3.2604687213897705, 10.458905220031738, 0) # Also represents: N33, N43
model.add_node('N32', 3.2604687213897705, 10.458905220031738, 5) # Also represents: N34, N47
model.add_node('N35', 1.2679941654205322, 10.458905220031738, 0) # Also represents: N37, N44
model.add_node('N36', 1.2679941654205322, 10.458905220031738, 5) # Also represents: N38, N48
model.add_node('N49', 3.2604687213897705, 5.641730785369873, 0) # Also represents: N63, N65, N83, N85, N92
model.add_node('N50', 3.2604687213897705, 5.641730785369873, 5) # Also represents: N64, N69, N84, N86, N96
model.add_node('N51', 6.271378040313721, 5.641730785369873, 0) # Also represents: N53, N66
model.add_node('N52', 6.271378040313721, 5.641730785369873, 5) # Also represents: N54, N70
model.add_node('N55', 6.271378040313721, 8.650237083435059, 0) # Also represents: N57, N67
model.add_node('N56', 6.271378040313721, 8.650237083435059, 5) # Also represents: N58, N71
model.add_node('N59', 3.2604687213897705, 8.650237083435059, 0) # Also represents: N61, N68
model.add_node('N60', 3.2604687213897705, 8.650237083435059, 5) # Also represents: N62, N72
model.add_node('N73', 3.2604687213897705, 3.831695556640625, 0) # Also represents: N87, N89, N107, N109, N116, N127, N129, N139, N147, N149, N162
model.add_node('N74', 3.2604687213897705, 3.831695556640625, 5) # Also represents: N88, N93, N108, N110, N120, N128, N130, N143, N148, N150, N166
model.add_node('N75', 8.351167678833008, 3.831695556640625, 0) # Also represents: N77, N90
model.add_node('N76', 8.351167678833008, 3.831695556640625, 5) # Also represents: N78, N94
model.add_node('N79', 8.351167678833008, 5.641730785369873, 0) # Also represents: N81, N91
model.add_node('N80', 8.351167678833008, 5.641730785369873, 5) # Also represents: N82, N95
model.add_node('N97', 3.2604687213897705, 1.3467525243759155, 0) # Also represents: N111, N113
model.add_node('N98', 3.2604687213897705, 1.3467525243759155, 5) # Also represents: N112, N117
model.add_node('N99', 5.686288356781006, 1.3467525243759155, 0) # Also represents: N101, N114
model.add_node('N100', 5.686288356781006, 1.3467525243759155, 5) # Also represents: N102, N118
model.add_node('N103', 5.686288356781006, 3.831695556640625, 0) # Also represents: N105, N115
model.add_node('N104', 5.686288356781006, 3.831695556640625, 5) # Also represents: N106, N119
model.add_node('N121', 0, 0, 0) # Also represents: N135, N137
model.add_node('N122', 0, 0, 5) # Also represents: N136, N141
model.add_node('N123', 3.2604687213897705, 0, 0) # Also represents: N125, N138
model.add_node('N124', 3.2604687213897705, 0, 5) # Also represents: N126, N142
model.add_node('N131', 0, 3.831695556640625, 0) # Also represents: N133, N140, N145, N159, N161
model.add_node('N132', 0, 3.831695556640625, 5) # Also represents: N134, N144, N146, N160, N165
model.add_node('N155', 0, 6.841569900512695, 0) # Also represents: N157, N164, N175, N177, N187
model.add_node('N156', 0, 6.841569900512695, 5) # Also represents: N158, N168, N176, N178, N191
model.add_node('N169', -1.7790435552597046, 4.971345901489258, 0) # Also represents: N183, N185
model.add_node('N170', -1.7790435552597046, 4.971345901489258, 5) # Also represents: N184, N189
model.add_node('N171', 0, 4.971345901489258, 0) # Also represents: N173, N186, N199, N201, N211
model.add_node('N172', 0, 4.971345901489258, 5) # Also represents: N174, N190, N200, N202, N215
model.add_node('N179', -1.7790435552597046, 6.841569900512695, 0) # Also represents: N181, N188
model.add_node('N180', -1.7790435552597046, 6.841569900512695, 5) # Also represents: N182, N192
model.add_node('N193', -2.560492515563965, 1.9951889514923096, 0) # Also represents: N207, N209
model.add_node('N194', -2.560492515563965, 1.9951889514923096, 5) # Also represents: N208, N213
model.add_node('N195', 0, 1.9951889514923096, 0) # Also represents: N197, N210
model.add_node('N196', 0, 1.9951889514923096, 5) # Also represents: N198, N214
model.add_node('N203', -2.560492515563965, 4.971345901489258, 0) # Also represents: N205, N212
model.add_node('N204', -2.560492515563965, 4.971345901489258, 5) # Also represents: N206, N216

# Print summary
print(f"Created 54 nodes with unique coordinates out of 216 total node entries")

# Create a mapping dictionary for future reference
node_mapping = {
    # Original nodes mapped to the nodes actually created in the model
    'N1': 'N1', 'N15': 'N1', 'N17': 'N1',
    'N2': 'N2', 'N16': 'N2', 'N21': 'N2',
    'N3': 'N3', 'N5': 'N3', 'N18': 'N3', 'N25': 'N3', 'N39': 'N3', 'N41': 'N3',
    'N4': 'N4', 'N6': 'N4', 'N22': 'N4', 'N26': 'N4', 'N40': 'N4', 'N45': 'N4',
    # Additional mappings would continue...
}

# Example of how to continue with the model
print("\nNode creation complete. To proceed with your model:")
print("1. Add sections (beams, columns)")
print("2. Create members between nodes")
print("3. Define supports")
print("4. Apply loads")
print("5. Define load combinations")
print("6. Run analysis")

# For visualization, you could export to a format like this:
print("\nNode coordinates for visualization:")
for node_name, node_obj in model.nodes.items():
    print(f"{node_name}: ({node_obj.X}, {node_obj.Y}, {node_obj.Z})")

ModuleNotFoundError: No module named 'PyNite'

In [ ]:
from Pynite import FEModel3D

# Create a new finite element model
model = FEModel3D()

# Step 1: Define materials
model.add_material('Steel', E=29000, G=11500, nu=0.3, rho=0.284)

# Step 2: Define cross-sections
model.add_section('TopChord', A=12, Iz=150, Iy=150, J=15)
model.add_section('BotChord', A=10, Iz=120, Iy=120, J=12)
model.add_section('VertMember', A=8, Iz=80, Iy=80, J=8)
model.add_section('DiagMember', A=6, Iz=60, Iy=60, J=6)
model.add_section('Stringer', A=7, Iz=70, Iy=40, J=7)
model.add_section('CrossBeam', A=9, Iz=90, Iy=45, J=9)

# Step 3: Create nodes for a truss bridge

# Bridge dimensions
length = 100  # feet
width = 20    # feet
height = 15   # feet
num_panels = 10

panel_length = length / num_panels

# Create bottom chord nodes (left truss)
for i in range(num_panels + 1):
    x = i * panel_length
    model.add_node(f'BL{i}', x, 0, 0)

# Create bottom chord nodes (right truss)
for i in range(num_panels + 1):
    x = i * panel_length
    model.add_node(f'BR{i}', x, width, 0)

# Create top chord nodes (left truss)
for i in range(num_panels + 1):
    x = i * panel_length
    # Parabolic top chord (highest at the middle)
    z = height
    if i != num_panels // 2 and i != (num_panels // 2) + (num_panels % 2):
        # Calculate height using a parabolic equation
        # This creates a curved profile with maximum height at the middle
        z = height * (1 - ((2 * i / num_panels) - 1) ** 2)
    model.add_node(f'TL{i}', x, 0, z)

# Create top chord nodes (right truss)
for i in range(num_panels + 1):
    x = i * panel_length
    # Parabolic top chord (highest at the middle)
    z = height
    if i != num_panels // 2 and i != (num_panels // 2) + (num_panels % 2):
        # Calculate height using a parabolic equation
        z = height * (1 - ((2 * i / num_panels) - 1) ** 2)
    model.add_node(f'TR{i}', x, width, z)

# Create cross-beam nodes (adding interior nodes for more complexity)
num_cross_beams = 5  # Number of cross members per panel
for i in range(num_panels + 1):
    x = i * panel_length
    for j in range(1, num_cross_beams):
        y = j * (width / num_cross_beams)
        model.add_node(f'C{i}_{j}', x, y, 0)

# Create some additional intermediate nodes for more complexity
# Add nodes at quarter points along verticals
for i in range(num_panels + 1):
    x = i * panel_length
    
    # Left vertical intermediate nodes
    z_tl = model.nodes[f'TL{i}'].Z
    model.add_node(f'VL{i}_1', x, 0, z_tl * 0.25)
    model.add_node(f'VL{i}_2', x, 0, z_tl * 0.5)
    model.add_node(f'VL{i}_3', x, 0, z_tl * 0.75)
    
    # Right vertical intermediate nodes
    z_tr = model.nodes[f'TR{i}'].Z
    model.add_node(f'VR{i}_1', x, width, z_tr * 0.25)
    model.add_node(f'VR{i}_2', x, width, z_tr * 0.5)
    model.add_node(f'VR{i}_3', x, width, z_tr * 0.75)

# Add some diagonal bracing nodes in the central plane
for i in range(num_panels):
    x1 = i * panel_length
    x2 = (i + 1) * panel_length
    model.add_node(f'D{i}_1', (x1 + x2) / 2, width / 4, height / 3)
    model.add_node(f'D{i}_2', (x1 + x2) / 2, 3 * width / 4, height / 3)
    model.add_node(f'D{i}_3', (x1 + x2) / 2, width / 4, 2 * height / 3)
    model.add_node(f'D{i}_4', (x1 + x2) / 2, 3 * width / 4, 2 * height / 3)

# Step 4: Create members

# Create bottom chord members (left and right trusses)
for i in range(num_panels):
    # Left bottom chord
    model.add_member(f'BLC{i}', f'BL{i}', f'BL{i+1}', 'Steel', 'BotChord')
    # Right bottom chord
    model.add_member(f'BRC{i}', f'BR{i}', f'BR{i+1}', 'Steel', 'BotChord')

# Create top chord members (left and right trusses)
for i in range(num_panels):
    # Left top chord
    model.add_member(f'TLC{i}', f'TL{i}', f'TL{i+1}', 'Steel', 'TopChord')
    # Right top chord
    model.add_member(f'TRC{i}', f'TR{i}', f'TR{i+1}', 'Steel', 'TopChord')

# Create vertical members with intermediate nodes
for i in range(num_panels + 1):
    # Left truss verticals (with intermediate nodes)
    model.add_member(f'VL{i}_A', f'BL{i}', f'VL{i}_1', 'Steel', 'VertMember')
    model.add_member(f'VL{i}_B', f'VL{i}_1', f'VL{i}_2', 'Steel', 'VertMember')
    model.add_member(f'VL{i}_C', f'VL{i}_2', f'VL{i}_3', 'Steel', 'VertMember')
    model.add_member(f'VL{i}_D', f'VL{i}_3', f'TL{i}', 'Steel', 'VertMember')
    
    # Right truss verticals (with intermediate nodes)
    model.add_member(f'VR{i}_A', f'BR{i}', f'VR{i}_1', 'Steel', 'VertMember')
    model.add_member(f'VR{i}_B', f'VR{i}_1', f'VR{i}_2', 'Steel', 'VertMember')
    model.add_member(f'VR{i}_C', f'VR{i}_2', f'VR{i}_3', 'Steel', 'VertMember')
    model.add_member(f'VR{i}_D', f'VR{i}_3', f'TR{i}', 'Steel', 'VertMember')

# Create diagonal members in each panel (left and right trusses)
for i in range(num_panels):
    # Left truss diagonals
    if i % 2 == 0:  # Alternating pattern
        model.add_member(f'DL{i}', f'BL{i}', f'TL{i+1}', 'Steel', 'DiagMember')
    else:
        model.add_member(f'DL{i}', f'TL{i}', f'BL{i+1}', 'Steel', 'DiagMember')
    
    # Right truss diagonals
    if i % 2 == 0:  # Alternating pattern
        model.add_member(f'DR{i}', f'BR{i}', f'TR{i+1}', 'Steel', 'DiagMember')
    else:
        model.add_member(f'DR{i}', f'TR{i}', f'BR{i+1}', 'Steel', 'DiagMember')

# Create cross beams at panel points
for i in range(num_panels + 1):
    # Main cross beams connecting left and right trusses
    model.add_member(f'CB{i}', f'BL{i}', f'BR{i}', 'Steel', 'CrossBeam')
    
    # Additional cross beams with interior nodes
    for j in range(1, num_cross_beams):
        prev_node = f'BL{i}' if j == 1 else f'C{i}_{j-1}'
        next_node = f'C{i}_{j}'
        model.add_member(f'CB{i}_{j}a', prev_node, next_node, 'Steel', 'CrossBeam')
        
        prev_node = f'C{i}_{j}'
        next_node = f'BR{i}' if j == num_cross_beams - 1 else f'C{i}_{j+1}'
        model.add_member(f'CB{i}_{j}b', prev_node, next_node, 'Steel', 'CrossBeam')

# Create stringers (longitudinal floor beams)
num_stringers = num_cross_beams - 1
for j in range(1, num_cross_beams):
    for i in range(num_panels):
        model.add_member(f'S{i}_{j}', f'C{i}_{j}', f'C{i+1}_{j}', 'Steel', 'Stringer')

# Create additional diagonal bracing in the central plane
for i in range(num_panels):
    # Connect the diagonal bracing nodes
    model.add_member(f'DB{i}_1', f'D{i}_1', f'D{i}_2', 'Steel', 'DiagMember')
    model.add_member(f'DB{i}_2', f'D{i}_3', f'D{i}_4', 'Steel', 'DiagMember')
    model.add_member(f'DB{i}_3', f'D{i}_1', f'D{i}_3', 'Steel', 'DiagMember')
    model.add_member(f'DB{i}_4', f'D{i}_2', f'D{i}_4', 'Steel', 'DiagMember')
    
    # Connect to main truss nodes
    if i < num_panels - 1:
        model.add_member(f'DB{i}_5', f'D{i}_1', f'VL{i+1}_2', 'Steel', 'DiagMember')
        model.add_member(f'DB{i}_6', f'D{i}_2', f'VR{i+1}_2', 'Steel', 'DiagMember')
        model.add_member(f'DB{i}_7', f'D{i}_3', f'TL{i+1}', 'Steel', 'DiagMember')
        model.add_member(f'DB{i}_8', f'D{i}_4', f'TR{i+1}', 'Steel', 'DiagMember')

# Step 5: Define supports
# Add pin supports at one end, roller supports at the other
model.def_support('BL0', True, True, True, False, False, False)  # Pin
model.def_support('BR0', True, True, True, False, False, False)  # Pin
model.def_support(f'BL{num_panels}', False, True, True, False, False, False)  # Roller
model.def_support(f'BR{num_panels}', False, True, True, False, False, False)  # Roller

# Step 6: Apply loads
# Add uniform load to all stringers to simulate deck loading
for j in range(1, num_cross_beams):
    for i in range(num_panels):
        model.add_member_dist_load(f'S{i}_{j}', 'FZ', -1, -1, 0, panel_length, 'Dead Load')

# Add point loads at quarter points to simulate vehicular loading
quarter_point = num_panels // 4
three_quarter_point = 3 * num_panels // 4

for j in range(1, num_cross_beams):
    model.add_node_load(f'C{quarter_point}_{j}', 'FZ', -5, 'Live Load')
    model.add_node_load(f'C{three_quarter_point}_{j}', 'FZ', -5, 'Live Load')

# Add horizontal loads to simulate wind loading
for i in range(num_panels + 1):
    model.add_node_load(f'TL{i}', 'FY', 0.5, 'Wind Load')
    model.add_node_load(f'TR{i}', 'FY', -0.5, 'Wind Load')

# Step 7: Define load combinations
model.add_load_combo('1.2D+1.6L', {'Dead Load': 1.2, 'Live Load': 1.6})
model.add_load_combo('1.2D+1.0L+0.8W', {'Dead Load': 1.2, 'Live Load': 1.0, 'Wind Load': 0.8})

# Step 8: Analyze the model
print("Analyzing model...")
model.analyze_linear()
print("Analysis complete!")

# Step 9: Print summary statistics
print("\n=== MODEL STATISTICS ===")
print(f"Total Nodes: {len(model.nodes)}")
print(f"Total Members: {len(model.members)}")
print(f"Truss Length: {length} ft")
print(f"Truss Width: {width} ft")
print(f"Truss Height: {height} ft")
print(f"Number of Panels: {num_panels}")

# Step 10: Print some sample results
print("\n=== SAMPLE RESULTS ===")

# Print displacements at midspan
mid_node = f'BL{num_panels//2}'
dx = model.nodes[mid_node].DX['1.2D+1.6L']
dy = model.nodes[mid_node].DY['1.2D+1.6L']
dz = model.nodes[mid_node].DZ['1.2D+1.6L']
print(f"\nDisplacements at midspan node {mid_node} (combo 1.2D+1.6L):")
print(f"  X-displacement: {dx:.4f} inches")
print(f"  Y-displacement: {dy:.4f} inches")
print(f"  Z-displacement: {dz:.4f} inches")

# Print forces in a middle chord member
mid_member = f'BLC{num_panels//2}'
axial = model.members[mid_member].axial(panel_length/2, '1.2D+1.6L')
shear_y = model.members[mid_member].shear('Fy', panel_length/2, '1.2D+1.6L')
shear_z = model.members[mid_member].shear('Fz', panel_length/2, '1.2D+1.6L')
moment_y = model.members[mid_member].moment('My', panel_length/2, '1.2D+1.6L')
moment_z = model.members[mid_member].moment('Mz', panel_length/2, '1.2D+1.6L')
print(f"\nMember forces at midspan of {mid_member} (combo 1.2D+1.6L):")
print(f"  Axial force: {axial:.2f} kips")
print(f"  Shear Y: {shear_y:.2f} kips")
print(f"  Shear Z: {shear_z:.2f} kips")
print(f"  Moment Y: {moment_y:.2f} kip-ft")
print(f"  Moment Z: {moment_z:.2f} kip-ft")

# Print reactions at a support
support_node = 'BL0'
rx = model.nodes[support_node].RX['1.2D+1.6L']
ry = model.nodes[support_node].RY['1.2D+1.6L']
rz = model.nodes[support_node].RZ['1.2D+1.6L']
print(f"\nReactions at support {support_node} (combo 1.2D+1.6L):")
print(f"  X-reaction: {rx:.2f} kips")
print(f"  Y-reaction: {ry:.2f} kips")
print(f"  Z-reaction: {rz:.2f} kips")

TypeError: 'module' object is not callable